In [ ]:
# !pip install selenium==4.12.0
"""
To link with webdriver use the following in PowerShell:

& "C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\ChromeDebug"

Make sure you have closed all Chrome tabs before running this line
"""

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import time
import random
import csv

import os

try:
    options = webdriver.ChromeOptions()
    options.add_experimental_option("debuggerAddress", "localhost:9222")

    driver = webdriver.Chrome(options=options)
    actions = ActionChains(driver)

    print("Connected to Chrome")
    
except:
    print("Not connected to Chrome")


In [ ]:
def save_tiktok_videos(output):
    with open(output, "a", newline="") as f:
        writer = csv.writer(f)

        i=0
        while True:
            i += 1
            time.sleep(random.uniform(1.5, 3.0))

            # URL link
            url = driver.current_url.split("?")[0]

            # Likes
            try:
                likes = driver.find_element("css selector", '[data-e2e="browse-like-count"]').text
            except:
                likes = "N/A"

            # Comments
            try:
                comments = driver.find_element("css selector", '[data-e2e="browse-comment-count"]').text
            except:
                comments = "N/A"

            # Saves
            try:
                saves = driver.find_element("css selector", '[data-e2e="undefined-count"]').text
            except:
                saves = "N/A"

            # Expand description if possible
            # This is possible when the description is too long to show in one time
            try:
                more_button = driver.find_element("css selector", 'button.css-fmfe6r-0be0dc34--ButtonExpand.e1azxgbm2')
                more_button.click()
                time.sleep(random.uniform(0.5, 1.0)) 
            except:
                pass

            """
            The descriptions are separated in two parts. One part
            is without hashtags and the other part is with hashtags
            """
            # Build full description in correct order
            try:
                container = driver.find_element("css selector", '[data-e2e="browse-video-desc"]')
                elements = container.find_elements("css selector", '[data-e2e="new-desc-span"], [data-e2e="search-common-link"]')

                ordered_parts = []
                for el in elements:
                    text = el.text.strip()
                    if text != "":
                        ordered_parts.append(text)

                full_description_ordered = " ".join(ordered_parts)

            except:
                full_description_ordered = "N/A"

            print(f"Video {i} Likes count: {likes} | Comment count: {comments} | Saves count: {saves}")
            print(f"Link: {url}")
            print(f"Description: {full_description_ordered}")

            writer.writerow([likes, comments, saves, url, " " + full_description_ordered])

            url_before = driver.current_url

            actions.send_keys(Keys.ARROW_DOWN).perform()
            time.sleep(random.uniform(1.5, 2.5))

            url_after = driver.current_url
            if url_before == url_after:
                print("End of video list")
                break

    print("Finished!")
    return output


In [ ]:
def ensure_dir_for_file(filepath):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)

In [ ]:
output = "Here the output file"
ensure_dir_for_file(output)
save_tiktok_videos(output)